# Контекст

-  Найти в искомом датасете вопрос, которые похож на мой вопрос

# Чтение данных

In [28]:
import warnings
from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cdist
import numpy as np

warnings.filterwarnings("ignore")

In [17]:
with open('texts.txt', 'r') as file:
    data = file.readlines()

data = [i[:-2] + ' ?' for i in data]
data[:10]

['О чем дневник Анны Франк ?',
 'Каким будет приложение Anne Frank App ?',
 'Какая информация войдет в новой мобильное приложение ?',
 'Где скрывались члены семьи Франк и другие евреи ?',
 'Как Анна называла место, где она и ее семья скрывались от нацистов ?',
 'Как будет называться приложение, посвященное Анне Франк и включающее фрагменты ее дневника ?',
 'Когда в первый раз была опубликована книга с записями Анны Франк? ?',
 'На сколько языков переведён дневник ?',
 'Кто озвучивал фрагменты книжки с записями, впервые опубликованной в 1947 году ?',
 'Где родилась автор книги, на основе которой издательство Viking Press выпустит мобильное приложение ?']

# Скачивание модели и кодировка вопросов

In [ ]:
model = SentenceTransformer('quora-distilbert-multilingual')

In [18]:
questions_embeddings = model.encode(data)

In [19]:
questions_embeddings.shape

(1813, 768)

# Создаем свои вопросы, похожие на представленные

In [21]:
# произвольные вопросы из датасета
data[10:15]

['Что анонсировала Студия Epic Games ?',
 'Когда вышла первая часть Gears of War ?',
 'Какие режиссерские работы Бена Аффлека вышли на экраны в 2007-2012 годах ?',
 'В каких фильмах снялся Бен Аффлек ?',
 'В какую организацию входят персонажи из комиксов DC ?']

In [22]:
# мои собственные вопросы
questions_similitary = [
    'Epic Games что-нибудь заанонсила ?',
    'Есть ли дата выхода Gears of War 1 ?',
    'Как много режиссерских работ у Бена Аффлека ?',
    'Во многих ли фильмах снялся Бен Аффлек ?',
    'Какое название у организации DC героев ?'
]

In [26]:
# кодировка моих вопросов
questions_similitary_emb = model.encode(questions_similitary)

In [27]:
questions_similitary_emb.shape

(5, 768)

In [ ]:
distances = cdist(questions_similitary_emb, questions_similitary, "cosine")
distances

In [39]:
slovar = dict(zip(questions_similitary, questions_similitary_emb))
slovar

{'Epic Games что-нибудь заанонсила ?': array([ 7.13932440e-02,  5.76326288e-02,  3.79860729e-01,  3.15335721e-01,
         3.95294577e-02, -7.61578828e-02, -3.92949253e-01, -1.93826467e-01,
        -1.21448293e-01,  8.61071348e-02,  4.94938642e-02,  5.28790474e-01,
         1.49489015e-01,  1.59097895e-01,  2.29132190e-01,  2.69263536e-01,
        -4.93706644e-01, -2.02644756e-03,  1.95915878e-01, -4.62000847e-01,
        -2.27164373e-01,  2.98034221e-01,  1.12820037e-01,  1.99905291e-01,
        -3.34009230e-01,  4.38505501e-01,  1.55805171e-01, -2.39687115e-01,
        -2.35155031e-01,  2.01646566e-01, -1.21864431e-01,  4.72093433e-01,
        -4.73314151e-02, -5.47882974e-01,  6.70820400e-02,  8.20753723e-02,
        -6.37362525e-02,  3.86579297e-02,  2.84745097e-01, -9.13873897e-04,
         3.40138793e-01,  1.73924714e-01,  3.40623707e-01,  1.90548360e-01,
        -5.18792868e-02,  3.13847423e-01, -1.90073788e-01,  4.13867503e-01,
        -1.57732636e-01,  2.95475535e-02,  1.18514

In [51]:
# cdist предназначена  для  вычисления  попарных  расстояний  между  точками  в  многомерном  пространстве.
# в нашем случае cdist вычисляет растояние между моим вопросом и каждым вопросом в датасете
# float число это и есть расстояние между моим вопросом и вопросом из датасета
# Чем меньше число, тем ближе вопросы 
for q, qe in zip(questions_similitary, questions_similitary_emb):
    distances = cdist([qe], questions_embeddings, "cosine")[0]
    ind = np.argsort(distances, axis=0)[:5]
    print("\n Test Question:")
    print('_________________________________')
    print(q)
    print('_________________________________')
    for i,(dis,text) in enumerate(zip(distances[ind], [data[i] for i in ind])):
        print(dis,ind[i],text, sep="\t")


 Test Question:
_________________________________
Epic Games что-нибудь заанонсила ?
_________________________________
0.015240087330512075	10	Что анонсировала Студия Epic Games ?
0.07613445316096401	1082	Какова основная сложность при создании игры по классическому произведению, по мнению продюсера Джоната Найта ?
0.08243251115811578	1081	Игру на основании какого произведения продюсер студии Visceral Games Джонатан Найт обдумывает несколько лет ?
0.09254603406243678	899	Что прослеживается в матче, о котором сложилось впечатление Уткина, как о "сомнительной " встрече ?
0.09391477980943341	922	Как сыграли хозяева соревнований в финале кубка Мира - 2016 ?

 Test Question:
_________________________________
Есть ли дата выхода Gears of War 1 ?
_________________________________
0.03479378978986569	11	Когда вышла первая часть Gears of War ?
0.07178541171536379	497	Где, по мнению Козовалова, не будет войны ?
0.08338968853646056	10	Что анонсировала Студия Epic Games ?
0.0835431907972124	1645	Г